In [24]:
import joblib
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load Posenet

In [10]:
interpreter = tf.lite.Interpreter("C:/Users/hjhhi/Desktop/Codes/lite-model_movenet_singlepose_lightning_tflite_float16_4.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()
output = interpreter.get_output_details()[0]  # Model has single output.
input = interpreter.get_input_details()[0]  # Model has single input.
interpreter.invoke()

In [27]:
def image_preprocess(path):
    image = cv2.imread(path)
    image= image.astype("uint8")
    image = cv2.resize(image, (192, 192), interpolation=cv2.INTER_NEAREST)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image= np.resize(image, (1, 192, 192, 3))
    return image

아래쪽의 이미지는 모비우스에서 가져오고 base64 디코딩 과정을 거쳐야함

In [28]:
image = image_preprocess("C:/Users/hjhhi/Desktop/Codes/51.jpg")

interpreter.set_tensor(input['index'], image)
interpreter.invoke()
result = interpreter.get_tensor(output_details[0]['index'])
result = result.reshape(51)

# Load image_classifier

In [29]:
image_classifier = joblib.load("./image_model.pkl")

In [34]:
image_proba = image_classifier.predict_proba([result])

In [35]:
image_proba

array([[0.01758501, 0.00950702, 0.57819063, 0.12256325, 0.05857313,
        0.08835219, 0.12522878]])

# Send final prediction to Mobius 

In [41]:
Final_pred=10000

In [43]:
import requests
import json

url = "http://203.253.128.177:7579/Mobius/SW_Hackaton/test"

data = {
    "m2m:cin": {
        "con": Final_pred
    }
}

headers = {
  'Accept': 'application/json',
  'X-M2M-RI': '12345',
  'X-M2M-Origin': 'SOrigin',
  'Content-Type': 'application/json;ty=4'
}



response = requests.request("POST", url, headers=headers, data=json.dumps(data))

print(response.text)


{"m2m:cin":{"rn":"4-20220916095142457","ty":4,"pi":"3-20220915105957489894","ri":"4-20220916095142458460","ct":"20220916T095142","lt":"20220916T095142","st":14,"et":"20240916T095142","cs":5,"con":"10000","cr":"SOrigin"}}


In [ ]:
import json
import requests
from requests.exceptions import HTTPError
import shortuuid

serverIP = "192.168.0.220"

ae = "test_ae_create"
cnt = "cnt_create"
con = [{"nm" : "조정용","enm" : "jycho" }]

data = {
    "m2m:cin": {
        "con": con
    }
}

if __name__ == "__main__":
    headers = {
        "Accept": "application/json",
        "X-M2M-RI": "req" + shortuuid.ShortUUID().random(length=10),
        "X-M2M-Origin": "/S" + ae,
        "Content-Type": "application/json;ty=4"
    }

    try:
        response = requests.request("post",
                                    "http://" + serverIP + ":7579/Mobius/" + ae + "/" + cnt + "?rcn=1",
                                    data=json.dumps(data),
                                    headers=headers,
                                    verify=False)

        print(response.text)

    except HTTPError as error:
        print(f"HTTP error occurred: {error}")
    except Exception as error:
        print(f"Other error occurred: {error}")